In [12]:
import tensorflow as tf
from tensorflow import keras
# import keras
# from keras import backend as K
import numpy as np
import cv2
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os
from PIL import Image
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [13]:
def generate_dataset():
    
    ''' Generate dataset from the csv files '''
    
    df = pd.read_csv('C:\\Users\\tanak\\Documents\\TamukaNongera\\WBS\\School\\Projects\\FinalProject\\Emoticall\\trainingsets.csv')
    
    train_samples = df[df['Usage']=="Training"]
    validation_samples = df[df["Usage"]=="PublicTest"]
    test_samples = df[df["Usage"]=="PrivateTest"]
    
    y_train = train_samples.emotion.astype(np.int32).values
    y_valid = validation_samples.emotion.astype(np.int32).values
    y_test = test_samples.emotion.astype(np.int32).values
     
    X_train =np.array([ np.fromstring(image, np.uint8, sep=" ").reshape((48,48)) for image in train_samples.pixels])
    X_valid =np.array([ np.fromstring(image, np.uint8, sep=" ").reshape((48,48)) for image in validation_samples.pixels])
    X_test =np.array([ np.fromstring(image, np.uint8, sep=" ").reshape((48,48)) for image in test_samples.pixels])
    
    
    return X_train, y_train, X_valid, y_valid, X_test, y_test

TRAIN FROM GPU

In [14]:
tf.config.list_physical_devices(device_type=None)
physical_devices = tf.config.list_physical_devices('GPU')
print("Num GPUs:", len(physical_devices))

Num GPUs: 1


In [10]:
tf.test.is_gpu_available(
    cuda_only=False, min_cuda_compute_capability=None
)

Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.


True

In [11]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 7848548293394171275
xla_global_id: -1
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 1429661286
locality {
  bus_id: 1
  links {
  }
}
incarnation: 8586921631607805947
physical_device_desc: "device: 0, name: NVIDIA GeForce 940MX, pci bus id: 0000:01:00.0, compute capability: 5.0"
xla_global_id: 416903419
]


TRAINING MODEL

In [15]:
def generate_model(learning_rate=0.001):
    
    with tf.device('/gpu:0'):  
        
        model = keras.models.Sequential()
        
        model.add(keras.layers.Conv2D(64,(3,3), input_shape=(48,48, 1), padding="same"))
        model.add(keras.layers.BatchNormalization())
        model.add(keras.layers.Activation('relu'))
        model.add(keras.layers.MaxPooling2D())
        model.add(keras.layers.Dropout(0.20))
        
        model.add(keras.layers.Conv2D(128,(5,5), padding='same'))
        model.add(keras.layers.BatchNormalization())
        model.add(keras.layers.Activation('relu'))
        model.add(keras.layers.MaxPooling2D())
        model.add(keras.layers.Dropout(0.20))

        model.add(keras.layers.Conv2D(512,(3,3), padding="same"))
        model.add(keras.layers.BatchNormalization())
        model.add(keras.layers.Activation('relu'))
        model.add(keras.layers.MaxPooling2D())
        model.add(keras.layers.Dropout(0.20))
        
        model.add(keras.layers.Conv2D(512,(3,3), padding="same"))
        model.add(keras.layers.BatchNormalization())
        model.add(keras.layers.Activation('relu'))
        model.add(keras.layers.MaxPooling2D())
        model.add(keras.layers.Dropout(0.25))
        
        model.add(keras.layers.Conv2D(256,(3,3), padding="same", activation='relu'))
        model.add(keras.layers.Conv2D(128,(3,3), padding='same', activation='relu'))
        model.add(keras.layers.MaxPooling2D())
        model.add(keras.layers.Dropout(0.25))
        
        #model.add(keras.layers.GlobalAveragePooling2D())
        model.add(keras.layers.Flatten())
        model.add(keras.layers.Dense(256))
        model.add(keras.layers.BatchNormalization())
        model.add(keras.layers.Activation('relu'))
        model.add(keras.layers.Dropout(0.5))
        
        model.add(keras.layers.Dense(512, activation='relu'))
        model.add(keras.layers.BatchNormalization())
        model.add(keras.layers.Activation('relu'))
        model.add(keras.layers.Dropout(0.5))
        
        model.add(keras.layers.Dense(7,activation='softmax'))
        
        model.compile(loss="sparse_categorical_crossentropy", optimizer=keras.optimizers.Adam(learning_rate=learning_rate) , metrics=['accuracy'])
        return model

GENERATE MODEL

In [16]:
if __name__=="__main__":
    
    #df = pd.read_csv("./fer2013/fer2013.csv")
    X_train, y_train, X_valid, y_valid, X_test, y_test =  generate_dataset()
    
    X_train = X_train.reshape((-1,48,48,1)).astype(np.float32)
    X_valid = X_valid.reshape((-1,48,48,1)).astype(np.float32)
    X_test = X_test.reshape((-1,48,48,1)).astype(np.float32)
    
    X_train_std = X_train/255.
    X_valid_std = X_valid/255.
    X_test_std = X_test/255.
    
    model = generate_model(0.01)
    with tf.device("/gpu:0"):
        history = model.fit(X_train_std, y_train,batch_size=128,epochs=35, validation_data=(X_valid_std, y_valid), shuffle=True)
        model.save("my_model.h5")

C:\Users\tanak\anaconda3\envs\tf\lib\site-packages\tensorflow\python\keras\optimizer_v2\optimizer_v2.py:374: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  warnings.warn(


Epoch 1/35
95/95 [==============================] - 51s 430ms/step - loss: 1.9821 - accuracy: 0.2422 - val_loss: 1.6078 - val_accuracy: 0.2887
Epoch 2/35
95/95 [==============================] - 37s 393ms/step - loss: 1.7307 - accuracy: 0.2664 - val_loss: 1.5899 - val_accuracy: 0.2957
Epoch 3/35
95/95 [==============================] - 37s 393ms/step - loss: 1.6616 - accuracy: 0.2700 - val_loss: 1.6058 - val_accuracy: 0.2957
Epoch 4/35
95/95 [==============================] - 37s 394ms/step - loss: 1.6019 - accuracy: 0.2819 - val_loss: 1.5747 - val_accuracy: 0.2998
Epoch 5/35
95/95 [==============================] - 37s 395ms/step - loss: 1.5712 - accuracy: 0.2990 - val_loss: 1.6369 - val_accuracy: 0.3061
Epoch 6/35
95/95 [==============================] - 38s 395ms/step - loss: 1.5247 - accuracy: 0.3292 - val_loss: 1.7504 - val_accuracy: 0.2126
Epoch 7/35
95/95 [==============================] - 38s 396ms/step - loss: 1.4698 - accuracy: 0.3523 - val_loss: 1.6036 - val_accuracy: 0.2938